# Library for neural networks

Feedforward neural networks for supervised learning

## Imports

In [1]:
from classes import Neuron, Layer, Network_Model
import helpers as hlp
import pandas as pd

## Preprocess input data

The `pandas`-Library is used to preprocess input data.
Firstly, data is read from an Excel-file.

In [15]:
# train_orig_df = pd.read_excel('datasets\\xor.xlsx')
train_orig_df = pd.read_excel('datasets\\rgb.xlsx')
train_df = train_orig_df.copy()
train_df.drop(columns=['Name'], inplace=True)
train_df

,R,G,B,Beige,Yellow,Orange,Red,Purple,Blue,Green,Grey,Brown,White,Black
0,138,102,66,1,0,0,0,0,0,0,0,0,0,0
1,169,131,7,1,0,0,0,0,0,0,0,0,0,0
2,190,189,127,1,0,0,0,0,0,0,0,0,0,0
3,194,176,120,1,0,0,0,0,0,0,0,0,0,0
4,198,166,100,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,244,169,0,0,1,0,0,0,0,0,0,0,0,0
209,245,208,51,0,1,0,0,0,0,0,0,0,0,0
210,248,243,53,0,1,0,0,0,0,0,0,0,0,0
211,250,210,1,0,1,0,0,0,0,0,0,0,0,0


The data is then split into training and validation data using the `split_training_val_data()` function. Here a percentage can be set.

In [8]:
train_df, val_df = hlp.split_training_val_data(train_df, 0.2)

The training and validation data are then split into input and output data using the `split_input_output_data()` function. The column headers of the training input have to specified.

In [9]:
# x_columns = ['A', 'B', 'C']
# y_columns = ['O']

x_columns = ['R', 'G', 'B']
y_columns = ['Beige', 'Yellow', 'Orange', 'Red', 'Purple', 'Blue', 'Green', 'Grey', 'Brown', 'White', 'Black']

train_df_x, train_df_y = hlp.split_input_output_data(train_df, x_columns, y_columns)
val_df_x, val_df_y = hlp.split_input_output_data(val_df, x_columns, y_columns)

In [10]:
train_df_x_norm = hlp.normalize(train_df_x, 1, -1)
val_df_x_norm = hlp.normalize(val_df_x, 1, -1)

## Instanciate a model

`add_layer` adds a new layer to the model. The amount of neurons and the desired activation function can be set. Layer 0 is automatically set to be the input layer. Weights and biases are randomly assigned.

In [2]:
mdl = Network_Model()
mdl.add_layer(3)
mdl.add_layer(16, activation='tanh')
mdl.add_layer(16, activation='tanh')
mdl.add_layer(11, activation='binary_step')

Using `plot_network` or `plot_network_pretty`, the Model can be visualized.

In [3]:
mdl.plot_network_pretty()

<div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(0, 0)<br>output 0<br>bias 0</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(0, 1)<br>output 0<br>bias 0</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(0, 2)<br>output 0<br>bias 0</div>

<div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 0)<br>output 0<br>bias -0.55</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 1)<br>output 0<br>bias -0.01</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 2)<br>output 0<br>bias -0.75</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 3)<br>output 0<br>bias 1.0</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 4)<br>output 0<br>bias -0.3</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 5)<br>output 0<br>bias -0.68</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 6)<br>output 0<br>bias -0.09</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 7)<br>output 0<br>bias 0.77</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 8)<br>output 0<br>bias 0.96</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 9)<br>output 0<br>bias 0.21</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 10)<br>output 0<br>bias -0.04</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 11)<br>output 0<br>bias 0.82</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 12)<br>output 0<br>bias 0.25</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 13)<br>output 0<br>bias 0.84</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 14)<br>output 0<br>bias -0.75</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 15)<br>output 0<br>bias 0.32</div>

<div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 0)<br>output 0<br>bias 0.36</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 1)<br>output 0<br>bias 0.99</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 2)<br>output 0<br>bias 0.33</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 3)<br>output 0<br>bias 0.03</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 4)<br>output 0<br>bias 0.86</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 5)<br>output 0<br>bias 0.39</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 6)<br>output 0<br>bias 0.55</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 7)<br>output 0<br>bias 0.35</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 8)<br>output 0<br>bias -0.08</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 9)<br>output 0<br>bias 0.47</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 10)<br>output 0<br>bias -0.97</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 11)<br>output 0<br>bias -0.67</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 12)<br>output 0<br>bias -0.8</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 13)<br>output 0<br>bias 0.74</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 14)<br>output 0<br>bias -0.89</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 15)<br>output 0<br>bias 0.6</div>

<div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 0)<br>output 0<br>bias -0.71</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 1)<br>output 0<br>bias 0.71</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 2)<br>output 0<br>bias 0.75</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 3)<br>output 0<br>bias -0.58</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 4)<br>output 0<br>bias 0.1</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 5)<br>output 0<br>bias 0.03</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 6)<br>output 0<br>bias -0.54</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 7)<br>output 0<br>bias 0.96</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 8)<br>output 0<br>bias -0.31</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 9)<br>output 0<br>bias 0.36</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 10)<br>output 0<br>bias 0.24</div>

In [12]:
pd.DataFrame(mdl.layers[1].weights)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.14,0.24,0.70,0.41,0.04,0.50,0.56,0.26,0.13,0.78,0.64,0.15,0.22,0.56,0.05,0.49
1,0.41,0.11,0.98,0.22,0.08,0.92,0.02,1.00,0.57,0.09,0.38,0.77,0.35,0.34,0.75,0.13
2,0.01,0.45,0.10,0.14,0.36,0.82,0.11,0.02,0.73,0.28,0.14,0.06,0.23,0.88,0.58,0.58


Using the `predict()` method, a prediction is made based on a given input-vector. Since the model is not trained yet, the output is basically random.

In [13]:
input = [1, 1, 0]
mdl.predict(input)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [14]:
mdl.plot_network_pretty()

<div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(0, 0)<br>output 1<br>bias 0</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(0, 1)<br>output 1<br>bias 0</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(0, 2)<br>output 0<br>bias 0</div>

<div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 0)<br>output 0.8<br>bias -0.55</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 1)<br>output 0.35<br>bias -0.01</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 2)<br>output 0.98<br>bias -0.75</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 3)<br>output -0.35<br>bias 1.0</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 4)<br>output 0.4<br>bias -0.3</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 5)<br>output 0.97<br>bias -0.68</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 6)<br>output 0.58<br>bias -0.09</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 7)<br>output 0.45<br>bias 0.77</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 8)<br>output -0.25<br>bias 0.96</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 9)<br>output 0.58<br>bias 0.21</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 10)<br>output 0.79<br>bias -0.04</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 11)<br>output 0.1<br>bias 0.82</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 12)<br>output 0.31<br>bias 0.25</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 13)<br>output 0.06<br>bias 0.84</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 14)<br>output 0.91<br>bias -0.75</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(1, 15)<br>output 0.29<br>bias 0.32</div>

<div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 0)<br>output 1.0<br>bias 0.36</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 1)<br>output 1.0<br>bias 0.99</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 2)<br>output 1.0<br>bias 0.33</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 3)<br>output 1.0<br>bias 0.03</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 4)<br>output 0.99<br>bias 0.86</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 5)<br>output 1.0<br>bias 0.39</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 6)<br>output 1.0<br>bias 0.55</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 7)<br>output 1.0<br>bias 0.35</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 8)<br>output 0.99<br>bias -0.08</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 9)<br>output 1.0<br>bias 0.47</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 10)<br>output 1.0<br>bias -0.97</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 11)<br>output 0.99<br>bias -0.67</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 12)<br>output 1.0<br>bias -0.8</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 13)<br>output 1.0<br>bias 0.74</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 14)<br>output 1.0<br>bias -0.89</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(2, 15)<br>output 0.98<br>bias 0.6</div>

<div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 0)<br>output 1<br>bias -0.71</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 1)<br>output 1<br>bias 0.71</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 2)<br>output 1<br>bias 0.75</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 3)<br>output 1<br>bias -0.58</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 4)<br>output 1<br>bias 0.1</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 5)<br>output 1<br>bias 0.03</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 6)<br>output 1<br>bias -0.54</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 7)<br>output 1<br>bias 0.96</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 8)<br>output 1<br>bias -0.31</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 9)<br>output 1<br>bias 0.36</div><div style="border-style:outset; border-radius: 1ex; border-color: white; padding: 0.5ex; text-align: center; float: left; margin: 0.25ex; width: fit-content">(3, 10)<br>output 1<br>bias 0.24</div>

## Training the model

In [11]:
mdl.train(train_df_x_norm, train_df_y, val_df_x_norm, val_df_y, )

p: [0.08235294117647052, -0.19999999999999996, -0.48235294117647054]
y: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
E_p:[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Err_p:0.5

p: [0.3254901960784313, 0.027450980392156765, -0.9450980392156862]
y: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
E_p:[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Err_p:0.5

p: [0.5215686274509803, 0.3803921568627451, -0.05882352941176472]
y: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
t: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
E_p:[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Err_p:5.0

p: [0.5607843137254902, 0.41176470588235303, -0.4509803921568627]
y: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
t: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
E_p:[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Err_p:5.0

p: [0.607843137254902, 0.28627450980392166, -0.592156862745098]
y: [1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]
t: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
E_p:[0, 0, 0, -1, -1, -1, 0, 0, -1, 0, 0]
Err_p:2.0

p: [0.7254901960784315, 0.223